In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

train_dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer/train.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer/test.csv")

In [6]:
train_datas = train_dataset.loc[:, train_dataset.columns != "label"].values / 255
train_labels = train_dataset["label"].values

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.optim as optim


train_x, test_x, train_y, test_y = train_test_split(train_datas, train_labels, test_size = 0.2, random_state = 22)

train_x = torch.from_numpy(train_x).type(torch.float32)
train_y = torch.from_numpy(train_y)
test_x = torch.from_numpy(test_x).type(torch.float32)
test_y = torch.from_numpy(test_y)

In [34]:
batch_size = 100
num_epochs = 10

train = torch.utils.data.TensorDataset(train_x, train_y)
test = torch.utils.data.TensorDataset(test_x, test_y)

train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [21]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1) # 28
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) # 14

        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1) # 14
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) # 7

        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1) # 14
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2) # 3

        self.fc1 = nn.Linear(64 * 3 * 3, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.cnn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [22]:
model = CNNModel()

if torch.cuda.is_available():
  model = model.cuda()

error = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

In [35]:
import time

start = time.time()
count = 0
best_accuracy = 0.0
print("-------Running-------")
for epoch in range(num_epochs):
    for (images, labels) in train_loader:
        
        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        optimizer.zero_grad()
        
        outputs = model(images.view(batch_size, 1, 28, 28))
        
        loss = error(outputs, labels)
        
        loss.backward()
        
        optimizer.step()

        count += 1
        with torch.no_grad():
          correct = 0
          total = 0

          predicted = torch.max(outputs.data, 1)[1]
          correct += (predicted == labels).sum()
          total += len(labels)

          train_accuracy = 100 * correct / float(total)
          if count % 10 == 0:
              # Calculate Accuracy         
              correct = 0
              total = 0
              # Iterate through test dataset
              for images, labels in test_loader:
                  if torch.cuda.is_available():
                    images = images.cuda()
                    labels = labels.cuda()
                  # Forward propagation
                  outputs = model(images.view(batch_size, 1, 28, 28))
                  
                  # Get predictions from the maximum value
                  predicted = torch.max(outputs.data, 1)[1]
          
                  # Total number of labels
                  total += len(labels)
                  correct += (predicted == labels).sum()
              
              valid_accuracy = 100 * correct/ float(total)
              if valid_accuracy > best_accuracy:
                  best_accuracy = valid_accuracy
                  best_model_state = model.state_dict()
                  best_optim_state = optimizer.state_dict()

    print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || [{epoch}/{num_epochs}], train_accuracy = {train_accuracy.data:.2f}, valid_accuracy = {valid_accuracy.data:.2f}""")
print("-------End------")
elapsed = time.time() - start
print(f"End of training, elapsed time : {elapsed // 60} min {elapsed % 60} sec.")

-------Running-------
2022-12-24 16:17:05 || [0/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:11 || [1/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:16 || [2/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:22 || [3/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:27 || [4/10], train_accuracy = 100.00, valid_accuracy = 98.89
2022-12-24 16:17:32 || [5/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:37 || [6/10], train_accuracy = 100.00, valid_accuracy = 98.92
2022-12-24 16:17:42 || [7/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:47 || [8/10], train_accuracy = 100.00, valid_accuracy = 98.90
2022-12-24 16:17:52 || [9/10], train_accuracy = 100.00, valid_accuracy = 98.90
-------End------
End of training, elapsed time : 0.0 min 52.037750005722046 sec.


In [32]:
print("The best validation accuracy was " + str(round(best_accuracy.item(), 3)) + " [%]")

The best validation accuracy was 99.012 [%]


In [33]:
filepath = "/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer"
modelpath = "model"
if best_model_state is not None and best_optim_state is not None:
    torch.save(best_model_state, f"{filepath}/{modelpath}/model_state_dict.pt")
    torch.save(best_optim_state, f"{filepath}/{modelpath}/optim_state_dict.pt")
    # model.load_state_dict(best_model_state)
    # torch.save(model, f"{filepath}/{experiment}/best_model.pt")
    print("Successfully saved.")

Successfully saved.
